# P3: Wrangle OpenStreetMap Data

## Data

The map area I chose is the Austin, TX area. As delineated in the class, I obtained the data by downloading an already-prepared extract which I found in the link below:

https://mapzen.com/data/metro-extracts/metro/austin_texas/

I chose the 66MB raw OpenStreetMap OSM XML dataset. After unzipping the file, it gave about 1.4 GB dataset. Opening this dataset using Sublime took a while.

### Preliminary examination of the dataset

This is done to see how the data looks like.

In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re

In [2]:
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

In [3]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type] += 1

In [4]:
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k,v)

In [5]:
def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

In [6]:
osmfile = "austin_texas_sample.osm"

In [7]:
for event, element in ET.iterparse(osmfile):
    if is_street_name(element):
        audit_street_type(street_types, element.attrib['v'])
print_sorted_dict(street_types)

#100: 1
#L2: 1
1100: 5
129: 1
1327: 5
1431: 10
150: 1
1625: 6
1626: 10
1826: 5
183: 1
2222: 8
2243: 1
2769: 17
290: 29
35: 2
620: 62
7: 1
71: 1
812: 14
973: 17
A: 5
Acres: 1
Affirmed: 1
Alley: 4
Alps: 3
Alto: 5
Amistad: 3
Arbolago: 2
Arrow: 1
Atlantic: 1
Ave: 3
Avenue: 1598
B: 8
Barrhead: 1
Bend: 180
Blackfoot: 2
Bluff: 4
Blvd: 3
Blvd.: 1
Bonanza: 3
Bonita: 4
Boulevard: 871
Branch: 2
Bridge: 3
C: 9
Calle: 2
Camino: 3
Cantera: 2
Canterwood: 3
Canyon: 7
Capri: 3
Casitas: 6
Castle: 5
CastlePath: 1
Catcher: 6
Catorce: 1
Cave: 5
Cc: 1
Challenger: 10
Chase: 12
Circle: 1077
Claro: 2
Claw: 1
Cliff: 2
Cobblestone: 8
Comanche: 1
Comet: 1
Corners: 8
Corral: 4
Costa: 1
Court: 788
Cove: 1648
cove: 2
Creek: 12
Crescent: 5
Criswell: 2
Crossing: 62
Ct: 6
Cutoff: 1
Cv: 24
D: 16
Dale: 2
Dalmahoy: 1
Dance: 9
Divide: 8
Dog: 2
Dorado: 9
Dr: 30
Dr.: 1
Dragon: 4
Drive: 11707
Dublin: 1
East: 48
Edenderry: 1
End: 1
Estates: 2
Explorer: 6
Expressway: 20
F: 32
Fairway: 3
Falcon: 3
Fields: 4
Firebird: 3
Fleur: 1


##### From above, we can see that there are street names that need to be fixed.

Avenue, Ave., Ave, and Avene

Boulevard, Blvd, Blvd.

Circle, Cc(?)

Costa, Corta(?)

Court, court, Ct

Cove, cove, Cv

Drive, Dr, Dr.

"Drive/Rd"?

Highway <= Hwy

FM1431, 1431, RM1431

I35, IH-35, IH35, IH35,

Lane, lane, Lanes(?), Ln

Pass, pass

Parkway, Pkwy

Place, Pl

Overlook, Ovlk

North, N(?)

Ps(?)

Road, Rd, "Road,1100"

SB?

St, St. street, Street

Trail, Tr, Trl

West, W

Way, way


## High Level Tags

To determine the number of high level tags the dataset has, iterative parsing is done.

In [8]:
import pprint

In [9]:
def count_tags(filename):
    tag_counts = defaultdict(int)
    for event, elem in ET.iterparse(filename):
        tag_counts[elem.tag] += 1
    return tag_counts

In [10]:
tags = count_tags(osmfile)

In [11]:
pprint.pprint(dict(tags))

{'member': 2218,
 'nd': 699761,
 'node': 635640,
 'osm': 1,
 'relation': 236,
 'tag': 237818,
 'way': 66639}


## Checking the k values

In [12]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [13]:
def key_type(element, keys):
    if element.tag == 'tag':
        try:
            lower.search(element.attrib['k']).group()
            keys["lower"] += 1
        except AttributeError:
            try:
                lower_colon.search(element.attrib['k']).group()
                keys["lower_colon"] += 1
            except AttributeError:
                try:
                    problemchars.search(element.attrib['k']).group()
                    keys["problemchars"] += 1
                except AttributeError:
                    keys["other"] += 1
    return keys

In [14]:
def process_map_tagtypes(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other":0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

In [15]:
keys = process_map_tagtypes(osmfile)

In [16]:
pprint.pprint(keys)

{'lower': 129788, 'lower_colon': 106850, 'other': 1180, 'problemchars': 0}


## Exploring Users

In [17]:
def get_user(element):
    return

In [18]:
def process_map_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for key in element.attrib:
            if key == 'uid':
                users.add(element.attrib[key])
    return users

In [19]:
users = process_map_users(osmfile)

In [20]:
len(users)

722

## Auditing and Improving Street Names 

Auditing the osmfile and using the variable 'mapping', check to see the changes needed to fix the unexpected street types to the appropriate ones in the expected list.

In [21]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

In [22]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Cove", "Highway", "Lane"]

In [23]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [24]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [25]:
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [26]:
st_types = audit(osmfile)

In [27]:
mapping = { "St": "Street",
            "St.": "Street",
            "st": "Street",
            "street": "Street",
            "Street,": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Avene": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Boulevard,": "Boulevard",
            "Blvd,": "Boulevard",
            "Blvd.,": "Boulevard",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Ct": "Court",
            "Ct.": "Court",
            "court": "Court",
            "Cv": "Cove",
            "cove": "Cove",
            "Cc": "Cove",
            "Pl": "Place",
            "Pl.": "Place",
            "lane": "Lane",
            "Ln": "Lane",
            "Rd": "Road", 
            "Rd.": "Road",
            "R": "Road",
            "Trl": "Trail",
            "Tr": "Trail",
            "Pkwy": "Parkway",
            "Hwy": "Highway",
            "HWY": "Highway",
            "Hwy,": "Highway",
            "H": "Highway",
            "I35": "Interstate Highway 35",
            "IH35": "Interstate Highway 35",
            "IH35,": "Interstate Highway 35",
            "IH-35": "Interstate Highway 35",
            "I-35": "Interstate Highway 35",
            "IH": "Interstate Highway",
            "I": "Interstate",
            "35,": "35",
            "main": "Main",
            "N": "North",
            "N.": "North",
            "Ovlk": "Overlook",
            "pass": "Pass",
            "Ps": "Pass",
            "W": "West",
            "W.": "West",
            "E": "East",
            "E.": "East",
            "texas": "Texas",
            "TX": "Texas",
            "FM": "Farm-to-Market Road",
            "F.M.": "Farm-to-Market Road",
            "U.S.": "United States",
            "US": "United States",
            "RM": "Ranch-to-Market Road",
            "RM1431": "Ranch-to-Market Road 1431",
            "RR": "Ranch Road",
            "S": "South",
            "S.": "South",
            "south": "South",
            "Bldg": "Building",
            "Bldg.": "Building",
            "Bld": "Building",
            "Ste": "Suite",
            "Ste,": "Suite",
            "STE": "Suite",
            "Ste.": "Suite",
            "suite": "Suite",
            "C": "Country",
            "church": "Church",
            "brigadoon": "Brigadoon",
            "Drive/Rd": "Drive/Road",
            "Mo-Pac": "MoPac", 
            "Avenue,Ste": "Avenue Suite",
            "suite#L131": "Suite L131"}

In [28]:
def append_highway(name):
    newparts = []
    parts = name.split()
    for item in parts:
        if (item == "Interstate" or item == "States") and "Highway" not in parts:
            newparts.append(item)
            newparts.append("Highway")
        else:
            newparts.append(item)
    new_name = ' '.join(newparts)
    return new_name

In [29]:
def update_farm_ranch_to_market(name):
    parts = name.split()
    if "Farm-to-Market" in parts or "Ranch-to-Market" in parts:
        if "Road" in parts:
            return name
        else:
            try:
                parts.insert(parts.index("Farm-to-Market")+1, "Road")
                name = " ".join(parts)
            except ValueError:
                parts.insert(parts.index("Ranch-to-Market")+1, "Road")  
                name = " ".join(parts)
    elif "Farm" in parts and "to" in parts and "Market" in parts:
        newname = []
        for i in range(parts.index("Farm")):
            newname.append(parts[parts.index(i)])
        newname.append("Farm-to-Market")
        newname += parts[parts.index("Market")+1:]

        if "Road" in parts:
            name = " ".join(newname)
        else:
            newname.insert(newname.index("Farm-to-Market")+1,"Road")
            name = " ".join(newname)
    
    elif "Ranch" in parts and "to" in parts and "Market" in parts:
        newname = []
        for i in range(parts.index("Ranch")):
            newname.append(parts[parts.index(i)])
        newname.append("Ranch-to-Market")
        newname += parts[parts.index("Market")+1:]

        if "Road" in parts:
            name = " ".join(newname)
        else:
            newname.insert(newname.index("Ranch-to-Market")+1,"Road")
            name = " ".join(newname)
        
    return name

In [30]:
def update_name(name, mapping):
    parts = name.split()
    newparts = []
    for item in parts:
        if item == "St" and "Rue" in parts:
            newparts.append("Saint")
        elif item == "N" or item == "C" or item == "I" or item == "H": 
            try:
                if newparts[0] == "Avenue":
                    newparts.append(item)
                else:
                    newparts.append(mapping[item])
            except IndexError:
                newparts.append(mapping[item])
        else:
            if item in mapping.keys():
                newparts.append(mapping[item])
            else:
                newparts.append(item)
    name = ' '.join(newparts)
    name = append_highway(name)
    name = update_farm_ranch_to_market(name)
    return name

In [31]:
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name

Merimac => Merimac
Melanie's Walk => Melanie's Walk
Winding Walk => Winding Walk
Gillians Walk => Gillians Walk
Whitemarsh Valley Walk => Whitemarsh Valley Walk
Midland Walk => Midland Walk
Julies Walk => Julies Walk
Village Walk => Village Walk
Liberty Walk => Liberty Walk
Regents Walk => Regents Walk
Mesa Ridge => Mesa Ridge
Panorama Ridge => Panorama Ridge
Ringtail Ridge => Ringtail Ridge
Points East Ridge => Points East Ridge
Ashton Ridge => Ashton Ridge
North Ridge => North Ridge
Honeycomb Ridge => Honeycomb Ridge
Riker Ridge => Riker Ridge
Creek Ridge => Creek Ridge
Carson Ridge => Carson Ridge
Hacienda Ridge => Hacienda Ridge
Aria Ridge => Aria Ridge
Smoky Ridge => Smoky Ridge
Sunset Ridge => Sunset Ridge
South Ridge => South Ridge
Musket Ridge => Musket Ridge
Cedar Ridge => Cedar Ridge
Runners Ridge => Runners Ridge
Pantera Ridge => Pantera Ridge
Timberline Ridge => Timberline Ridge
Buckskin Ridge => Buckskin Ridge
Crescent Bluff => Crescent Bluff
Travis Bluff => Travis Bluff
S

##### More things to fix:

- Ranch Road 620 is also Farm-to-Market 620. Which one is correct? I don't know. I'll just leave it as is.
- US highway 290 is also country road 290, etc. Will not fix that one either.

- postal code (even k value postal code varies (other are postcode))



## More exploration of data:

### Different values of k under tag

In [32]:
k = set()
for event, element in ET.iterparse(osmfile):
    if element.tag == "tag" or element.tag == "way":
        for tag in element.iter("tag"):
             k.add(tag.attrib['k'])

In [33]:
len(k)

347

In [34]:
k

{'FIXME',
 'NHS',
 'Texas_Trunk_System',
 'abandoned',
 'access',
 'addr:city',
 'addr:country',
 'addr:housename',
 'addr:housenumber',
 'addr:postcode',
 'addr:state',
 'addr:street',
 'addr:unit',
 'admin_level',
 'aeroway',
 'alt_name',
 'amenity',
 'area',
 'artist_name',
 'artwork_type',
 'atm',
 'attribution',
 'barrier',
 'basin',
 'bench',
 'bicycle',
 'bicycle_parking',
 'boat',
 'border_type',
 'boundary',
 'brand',
 'bridge',
 'bridge:name',
 'building',
 'building:levels',
 'building:min_level',
 'building_1',
 'bus',
 'button_operated',
 'cables',
 'capacity',
 'capacity:disabled',
 'capital',
 'car_wash',
 'cash',
 'cash_in',
 'census:population',
 'circuits',
 'coa:place_id',
 'colour',
 'commercial',
 'construction',
 'contact:phone',
 'covered',
 'craft',
 'created_by',
 'crop',
 'crossing',
 'cuisine',
 'currency:XBT',
 'cutting',
 'cycleway',
 'cycleway:foot',
 'cycleway:left',
 'cycleway:right',
 'delivery',
 'denomination',
 'description',
 'description:en',
 'des

#### Websites

In [35]:
for event, element in ET.iterparse(osmfile):
    if element.tag == "tag" or element.tag == "way":
        for tag in element.iter("tag"):
             if tag.attrib['k'] == 'website':
                    print tag.attrib['v']

http://www.heightsassembly.com/
http://texasmilitaryforcesmuseum.org/
http://www.epochcoffee.com/
http://gumbosaustin.com/gumbosroundrock.html
http://www.resistenciabooks.com/
http://local.randalls.com/tx/austin-2490.html
http://www.tienjinrestaurant.com/
http://www.shakespearesaustin.com/
http://www.tejifoods.com/teji/index.php
http://www.quiznos.com
http://minhscafe.menufy.com/
http://www.waterlooicehouse.com/
http://bravenewbookstore.com/
http://maithaionline.com/
http://www.casawatchers.com
http://tomosushiaustin.com/
http://www.discounttire.com
http://crossfitcitylimits.com/
http://www.nextgen.com/
http://atfcu.org/index.php/component/content/article/36-locations-a-hours/157-brodie-lane
http://mylocalbitcoinatm.com/
http://freshhealthycafeut.menufy.com/
http://sushidotcomaustin.com/
http://www.capmetro.org/gismaps/stops/5275.html
http://www.capmetro.org/gismaps/stops/5547.html
http://www.capmetro.org/gismaps/stops/5553.html
http://www.capmetro.org/gismaps/stops/5227.html
http://ww

In [36]:
def update_website(website):
    if website[:7] != "http://":
        website = "http://" + website
    return website

In [37]:
def is_website(tag):
    return element.attrib['k'] == "website"

#### Phone numbers

In [38]:
def is_phone(element):
    return (element.attrib['k'] == "phone" or element.attrib['k'] == "contact:phone")

In [64]:
def update_phone(number):
    phone_re = re.compile(r'^\d\d\d\-\d\d\d\-\d\d\d\d$')
    if phone_re.search(number) == None:
        phno = []
        number = list(number.lstrip("+1"))
        for char in number:
            try:
                if int(char) in [x for x in range(10)]:
                    phno.append(char)
            except ValueError:
                continue
        number = "".join(phno)
        number = number[:3] + "-" + number[3:6] + "-" + number[6:]
                    
    return number

In [62]:
phno = []
for char in list("+1 512-472-1666".strip("+1")):
    try:
        if int(char) in [x for x in range(10)]:
            phno.append(char)
    except ValueError:
        continue

In [63]:
phno

['5', '1', '2', '4', '7', '2', '1', '6', '6', '6']

In [65]:
for element in get_element(osmfile):
    if element.tag == "node":
        for tag in element.iter("tag"):
            if is_phone(tag):
                print tag.attrib['v'], "=>", update_phone(tag.attrib['v'])

512-444-0762 => 512-444-0762
(512) 782-5659 => 512-782-5659
512-338-2241 => 512-338-2241
(512) 892-6699 => 512-892-6699
+1 512-472-1666 => 512-472-1666
(512) 494-9300 => 512-494-9300
512-249-6464 => 512-249-6464
512-301-1007 => 512-301-1007
(512)482-8244 => 512-482-8244
888-619-2272 => 888-619-2272
+1-512-821-9472 => 512-821-9472
(512) 336-7200 => 512-336-7200
(512) 478-8086 => 512-478-8086
(512) 336-7706 => 512-336-7706
512-382-9106 => 512-382-9106
512-716-1118 => 512-716-1118
512-467-2225 => 512-467-2225
(512) 312-0907 => 512-312-0907
512-343-8020 => 512-343-8020
512-345-7238 => 512-345-7238
+1 512 357 4143 => 512-357-4143
512-447-6633 => 512-447-6633
51224990093 => 512-249-90093
+1 512-275-0852 => 512-275-0852
512-472-2783 => 512-472-2783
512-444-0561 => 512-444-0561
(512) 524-1311 => 512-524-1311
(512) 246-7645 => 512-246-7645
512-358-4379 => 512-358-4379


##### Country

In [66]:
def is_country(element):
    return (element.attrib['k'] == "addr:country")

In [67]:
country_re = re.compile(r'^US$')

In [68]:
country_re.search('US').group()

'US'

In [ ]:
#counter = 0
for element in get_element(osmfile):
#    if counter == 10:
#        break
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_country(tag):
                try:
                    country_re.search(tag.attrib['v']).group()
                except AttributeError:
                    print tag.attrib['v']
#                counter += 1

#### Postal Code

k = addr:postcode or postal_code

In [43]:
def is_postcode(element):
    return (element.attrib['k'] == "addr:postcode" or element.attrib['k'] == "postal_code")

In [46]:
#counter = 0
for element in get_element(osmfile):
#    if counter == 100:
#        break
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_postcode(tag):
                print tag.attrib['v']
#                counter += 1

78741
78703
78751
78681
78744
78751
78759
78745
78681
78757
78620
78749
78705
78727
78757
78753
78727
78748
78702
78726
78741
78741
78701
78701
78610
78664
78646
78750
78645
78640
78610
78731
78731
78702
78656
78745
78640
78759
78759
78759
78717
78702
78626
78704
78704
78704
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78705
78752
78752
78752
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78757
78731
78731
78731
78731
78731
78731
78731
78731
78731
78731
78731
78731
78731
78731
78757
78757
78757
78757
78757
78731
78731
78731
78731
78731
78731
78731
78731
78731
78757
78757
78757
78757
78751
78751
78751
78751
78731
78731
78751
78751
78751
78751
78751
78751
78757
78757
78757
78751
78751
78751
78731
78751
78751
78751
78751
78751
7875

When looking at the whole data, the postcode sometimes has the county codes in addition to the zip codes.

In [47]:
postcode_re = re.compile(r'^7\d\d\d\d$')

In [48]:
mo = postcode_re.search('78723')
mo.group()

'78723'

In [49]:
for element in get_element(osmfile):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_postcode(tag):
                try:
                    postcode_re.search(tag.attrib['v']).group()
                except AttributeError:
                    print tag.attrib['v']

78724-1199
tx


In [50]:
zipcounty_re = re.compile(r'^7\d\d\d\d\-\d\d\d\d$')

In [69]:
def update_postcode(postcode):
    postcode_re = re.compile(r'^7\d\d\d\d$')
    if postcode_re.search(postcode) == None:
        zipcounty_re = re.compile(r'^7\d\d\d\d\-\d\d\d\d$')
        if zipcounty_re.search(postcode) != None:
            postcode = postcode.split('-')[0]
            #countycode = postcode.split('-')[1]
        else:
            for item in postcode.split():
                if postcode_re.search(item) != None:
                    postcode = item
                else:
                    postcode = None
    return postcode

In [52]:
clean_postcode('TX 78724')

'78724'

In [ ]:
counter = 0
for element in get_element(osmfile):
    if counter == 30:
        break
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_postcode(tag):
                print tag.attrib['v'], "=>", clean_postcode(tag.attrib['v'])
                counter += 1

In [ ]:
clean_postcode('78626‎')

## Preparing for Database

In [53]:
import csv
import codecs
import cerberus
import schema

In [54]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

In [55]:
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [56]:
SCHEMA = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

Make sure the fields order in the csvs matches the column order in the sql table schema.

In [57]:
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS= ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [70]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, lower_colon=LOWER_COLON, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for field in node_attr_fields:
            node_attribs[field] = element.attrib[field]
        
        for tag in element.iter("tag"):
            nodetags = {}
            nodetags['id'] = element.attrib['id']
            
#            nodetags['value'] = tag.attrib['v']
            
            # Adding the nodetags['value'], and if it's a street name, clean it using update_name:
            if is_street_name(tag):
                nodetags['value'] = update_name(tag.attrib['v'], mapping)
            
            elif is_phone(tag):
                nodetags['value'] = update_phone(tag.attrib['v'])
            
            elif is_postcode(tag):
                nodetags['value'] = update_postcode(tag.attrib['v'])
                
            else: 
                nodetags['value'] = tag.attrib['v']
                
            # Adding the nodetags['type'] and nodetags['key'] and fixing them:
            try:
                problem_chars.search(tag.attrib['k']).group()
            except AttributeError:
                try:    
                    lower_colon.search(tag.attrib['k']).group()
                    kvalue = tag.attrib['k'].split(":")
                    nodetags['type'] = kvalue[0]
                    if len(kvalue) == 2:                        
                        nodetags['key'] = kvalue[1]
                    else:
                        nodetags['key'] = ':'.join(kvalue[1:])
                except AttributeError:
                    nodetags['type'] = default_tag_type
                    nodetags['key'] = tag.attrib['k']
            tags.append(nodetags) 
    
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for item in WAY_FIELDS:
            way_attribs[item] = element.attrib[item]
        
        for wtag in element.iter("tag"):
            waytags = {}
            waytags['id'] = element.attrib['id']
            
#            waytags['value'] = wtag.attrib['v']

            
            if is_street_name(wtag):
                waytags['value'] = update_name(wtag.attrib['v'], mapping)
                
            elif is_phone(wtag):
                waytags['value'] = update_phone(wtag.attrib['v'])
                
            elif is_postcode(wtag):
                waytags['value'] = update_postcode(wtag.attrib['v'])
                
            else:
                waytags['value'] = wtag.attrib['v']
            
            try:
                problem_chars.search(wtag.attrib['k']).group()
            except AttributeError:
                try:
                    lower_colon.search(wtag.attrib['k']).group()
                    wkvalue = wtag.attrib['k'].split(":")
                    waytags['type'] = wkvalue[0]
                    if len(wkvalue) == 2:
                        waytags['key'] = wkvalue[1]
                    else:
                        waytags['key'] = ':'.join(wkvalue[1:])
                except AttributeError:
                    waytags['type'] = default_tag_type
                    waytags['key'] = wtag.attrib['k']
            tags.append(waytags) 
        
        
        position = 0
        for waytag in element.iter("nd"):
            waynd = {}
            waynd['id'] = element.attrib['id']
            waynd['node_id'] = waytag.attrib['ref']
            waynd['position'] = position
            position += 1
            way_nodes.append(waynd)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [45]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [71]:
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )

In [72]:
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [76]:
def process_map_db(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, codecs.open(WAYS_PATH, 'w') as ways_file, codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

#### Processing the whole Austin, TX map:

In [79]:
process_map_db(osmfile, validate=True)

In [75]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, lower_colon=LOWER_COLON, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for field in node_attr_fields:
            node_attribs[field] = element.attrib[field]
        
        for tag in element.iter("tag"):
            nodetags = {}
            nodetags['id'] = element.attrib['id']
            nodetags['value'] = tag.attrib['v']
            try:
                problem_chars.search(tag.attrib['k']).group()
            except AttributeError:
                try:    
                    lower_colon.search(tag.attrib['k']).group()
                    kvalue = tag.attrib['k'].split(":")
                    nodetags['type'] = kvalue[0]
                    if len(kvalue) == 2:                        
                        nodetags['key'] = kvalue[1]
                    else:
                        nodetags['key'] = ':'.join(kvalue[1:])
                except AttributeError:
                    nodetags['type'] = default_tag_type
                    nodetags['key'] = tag.attrib['k']
            tags.append(nodetags) 
    
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for item in WAY_FIELDS:
            way_attribs[item] = element.attrib[item]
        
        for wtag in element.iter("tag"):
            waytags = {}
            waytags['id'] = element.attrib['id']
            waytags['value'] = wtag.attrib['v']
            try:
                problem_chars.search(wtag.attrib['k']).group()
            except AttributeError:
                try:
                    lower_colon.search(wtag.attrib['k']).group()
                    wkvalue = wtag.attrib['k'].split(":")
                    waytags['type'] = wkvalue[0]
                    if len(wkvalue) == 2:
                        waytags['key'] = wkvalue[1]
                    else:
                        waytags['key'] = ':'.join(wkvalue[1:])
                except AttributeError:
                    waytags['type'] = default_tag_type
                    waytags['key'] = wtag.attrib['k']
            tags.append(waytags) 
        
        
        position = 0
        for waytag in element.iter("nd"):
            waynd = {}
            waynd['id'] = element.attrib['id']
            waynd['node_id'] = waytag.attrib['ref']
            waynd['position'] = position
            position += 1
            way_nodes.append(waynd)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
